# Revised case normalization for Hirslanden Zurich 2019
note:
- Assert error: case_id in this hospital is 'Fall Nummer'
- merge from 78 to 78 -> 122 ???? due to duplicated cases in the revised cases
- '0044151336' and '44151336' both appears in the revised cases, using set() delted duplicated case_id



In [1]:
import pandas as pd
import sys
sys.path.insert(0, '/home/jovyan/work')

import os

from py.global_configs import *
from py.normalize import normalize

from src.service import bfs_cases_db_service as bfs_db

from src.utils.dataframe_utils import remove_duplicated_chops

/home/jovyan/work/src/models/BfsCase.py:5: SAWarning: On class 'BfsCase', Column object 'drg_cost_weight' named directly multiple times, only one will be used: drg_cost_weight, effective_cost_weight. Consider using orm.synonym instead
  class BfsCase(Base):


In [2]:

file_info = FileInfo(
        os.path.join(ROOT_DIR, 'raw_data/HI-Zurich.xlsx'),
        'Hirslanden Zurich',
         '2019',
        ['Änderungen_MIX_Hirslanden_2019'])

print(file_info)


FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Zurich.xlsx', hospital_name_db='Hirslanden Zurich', year='2019', sheets=['Änderungen_MIX_Hirslanden_2019'])


In [3]:
cols_to_rename = dict(COLUMNS_TO_RENAME)
# Replace 'admno' with 'fall nummer'
cols_to_rename.pop('admno')
cols_to_rename['fall nummer'] = CASE_ID_COL

df_revised_case_d2d = normalize(file_info, 0, columns_mapper = cols_to_rename)

2022-10-17 18:40:15.933 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 2: discarded ICDs after validation {''}
2022-10-17 18:40:15.934 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 4: discarded ICDs after validation {''}
2022-10-17 18:40:15.934 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 6: discarded ICDs after validation {''}
2022-10-17 18:40:15.935 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 8: discarded ICDs after validation {''}
2022-10-17 18:40:15.936 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 10: discarded ICDs after validation {''}
2022-10-17 18:40:15.936 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 12: discarded ICDs after validation {''}
2022-10-17 18:40:15.937 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 14: discarded ICDs after validation {''}
2022-10-17 18:40:15.938 | DEBUG    | src.utils.dataframe_utils:_validate_

Read 155 cases for Hirslanden Zurich 2019
77/155 rows were deleted because contained NaNs
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
old_pd              string
new_pd              string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object


# Match revised DtoD data with the bfs data from the database


In [4]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

array(['0044151336', '0044162952', '0044174916', '0044184703',
       '0044201740', '0044207672', '0044213833', '0044215776', '44101606',
       '44104525', '44105681', '44107335', '44107676', '44108722',
       '44109815', '44113655', '44125350', '44143612', '44162169',
       '44205344', '44212779', '44214368', '44107556', '44110727',
       '44118028', '44119597', '44125669', '44135768', '44139324',
       '44153256', '44156085', '44163377', '44181674', '44182926',
       '44084064', '44104968', '44107697', '44104968', '44125054',
       '44125625', '44134808', '44146114', '44152500', '0044191819',
       '44209519', '44116933', '44120153', '0044132051', '44164627',
       '0044176401', '44181751', '44184371', '44207478', '44101606',
       '44104525', '44105681', '44107335', '44107676', '44108722',
       '44109815', '44113655', '44125350', '44143612', '44162169',
       '44205344', '44212779', '44214368', '0044122622', '44128901',
       '44148694', '44151336', '44162952', '441749

In [5]:
revised_case_id_stripped = [case_id.lstrip('0') for case_id in revised_case_id]
len(revised_case_id_stripped)

78

### Check if there are duplicated case_ids, if yes, delete them

In [1]:
# Delete duplicated case_id in this hospital.

revised_case_id_stripped_set = set(revised_case_id_stripped)
if len(revised_case_id_stripped) > len(revised_case_id_stripped_set):
    print(f'There are duplicated case_ids in the revised case.' )

NameError: name 'revised_case_id_stripped' is not defined

In [7]:
# change the case_id the same format as bfs data e.g. '0041194601' 
# revised_case_id_pad0 = [case_id.zfill(10) for case_id in revised_case_id]
# revised_case_id_pad0[0]
# df_revised_case_d2d['case_id'] = revised_case_id_pad0

In [8]:
# match to the database and retrieve data from database using revised
revised_case_db = bfs_db.get_bfs_cases_by_ids_no_pad0(list(revised_case_id_stripped_set))
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl
0,1.940,234703,1,0044164627,58D35DE98034237D,88,0,M,5,4,0,0,0,2019-06-19,01,2019-06-24,00,F34D,F34,3
0,0.952,230910,1,0044134808,25AC80C8B5D88D22,61,0,M,4,4,0,0,0,2019-04-04,01,2019-04-08,00,F59F,F59,3
0,0.883,240439,1,0044212779,A56686572414BDAC,81,0,W,9,3,0,0,0,2019-10-06,01,2019-10-15,00,H41D,H41,2
0,1.940,228856,1,0044120153,25AC80C8B5D88D22,61,0,M,4,4,0,0,0,2019-02-17,01,2019-02-21,00,F34D,F34,3
0,0.658,229593,1,0044125350,0E25481D9EFB6C23,86,0,M,7,3,0,0,0,2019-02-25,01,2019-03-04,00,G46C,G46,2


In [9]:
# create a new column for case_id_stripped with '0' for joining to the revised cases
revised_case_db['case_id_stripped']=  revised_case_db['case_id'].apply(lambda x: x.lstrip('0')) 
revised_case_db.head()
print(revised_case_db.shape)

(55, 21)


In [10]:
df_revised_case_d2d['case_id_stripped']=  df_revised_case_d2d['case_id'].apply(lambda x: x.lstrip('0')) 
df_revised_case_d2d.head()
print(df_revised_case_d2d.shape)

(78, 15)


In [11]:
# delete duplicated cases in the revised cases

df_revised_case_d2d['case_id_stripped'].apply(tuple) # make sure the drop_duplicates works

df_revised_case_d2d.drop_duplicates(subset=['case_id_stripped'], keep='last', inplace=True)
assert len(revised_case_id_stripped) == df_revised_case_d2d.shape[0] # make sure all cases are deleted
print(f'The number of cases after deleting the duplicated cases is: {df_revised_case_d2d.shape[0]}')


# Forming the normalized dataframe: Merge the selected revised case data with the matched from bfs data
  If there are multiple cases were found based on case_id, patient_id will be used to match once more  



In [12]:
revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id', 'case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
revised_case_d2d_subset.head()

,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops,case_id,case_id_stripped,gender,age_years,duration_of_stay,pccl,drg
44,T821,T821,M200,[L8932],[],[],[],44107556,44107556,M,79,4,3,F18B
46,T828,T828,M200,"[D62, I350]",[],[],[],44110727,44110727,W,77,42,3,F34D
48,M4806,M4806,M200,[G2090],[],[],[],44118028,44118028,M,85,9,3,I10C
50,K660,K660,M000,[J841],[],[],[],44119597,44119597,M,71,7,3,G21B
52,N304,N304,M200,[T820],[],[],[],44125669,44125669,M,77,10,3,L04C


In [13]:
# Check if there are multiple cases were found based on case_id, patient_id will be used to match once more
if revised_case_db.shape[0] <= len(revised_case_id):
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_db.merge(revised_case_d2d_subset, how ='left', on=['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'])
    num_match_case_id = revised_case_normalized.shape[0]
else:
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_stripped', 'patient_id', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_db.merge(revised_case_d2d_subset, how ='left', on=['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'])
    num_match_case_patient_id = revised_case_normalized.shape[0]
print(revised_case_normalized.shape)
revised_case_normalized.head()

(55, 28)


,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,...,adrg,pccl,case_id_stripped,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,1.940,234703,1,0044164627,58D35DE98034237D,88,0,M,5,4,...,F34,3,44164627,I723,I723,M200,[I341],[],[],[]
1,0.952,230910,1,0044134808,25AC80C8B5D88D22,61,0,M,4,4,...,F59,3,44134808,T828,T828,M200,[E6682],[],[],[]
2,0.883,240439,1,0044212779,A56686572414BDAC,81,0,W,9,3,...,H41,2,44212779,K8510,K8510,M100,[I5013],[],[],[]
3,1.940,228856,1,0044120153,25AC80C8B5D88D22,61,0,M,4,4,...,F34,3,44120153,I7022,I7022,M200,[E6682],[],[],[]
4,0.658,229593,1,0044125350,0E25481D9EFB6C23,86,0,M,7,3,...,G46,2,44125350,K254,K254,M100,[D6835],[],[],[]


In [15]:
COLS_TO_OUTPUT

['aimedic_id',
 'case_id',
 'patient_id',
 'gender',
 'age_years',
 'duration_of_stay',
 'pccl',
 'drg',
 'old_pd',
 'new_pd',
 'bfs_code',
 'added_icds',
 'removed_icds',
 'added_chops',
 'removed_chops']

In [19]:
# add aimedic_id also in the output dataset, using COLS_TO_OUTPUT

df_revised_case_normalized_output = revised_case_normalized[COLS_TO_OUTPUT]
print(df_revised_case_normalized_output.shape)

df_revised_case_normalized_output.head()


(55, 15)


,aimedic_id,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,234703,0044164627,58D35DE98034237D,M,88,5,3,F34D,I723,I723,M200,[I341],[],[],[]
1,230910,0044134808,25AC80C8B5D88D22,M,61,4,3,F59F,T828,T828,M200,[E6682],[],[],[]
2,240439,0044212779,A56686572414BDAC,W,81,9,2,H41D,K8510,K8510,M100,[I5013],[],[],[]
3,228856,0044120153,25AC80C8B5D88D22,M,61,4,3,F34D,I7022,I7022,M200,[E6682],[],[],[]
4,229593,0044125350,0E25481D9EFB6C23,M,86,7,2,G46C,K254,K254,M100,[D6835],[],[],[]


# Export the normalized revised cases with the neccessary columns to a csv file


In [21]:

hospital = file_info.hospital_name_db.replace(' ', '_') # replace spaces with '_' for the file name
hospital_year = f'{hospital}_{file_info.year}'

output_folder = os.path.join(ROOT_DIR, 'normalized_revision_data/')

if not os.path.exists(output_folder):
    os.mkdir(output_folder)

output_path = os.path.join(ROOT_DIR, 'normalized_revision_data/') + hospital_year + '.csv'

df_revised_case_normalized_output.to_csv(output_path)


# Report

A small summary of:
- how many cases could be matched in the DB, and based on which columns
- how many cases were “correctly” revised, in terms of upcoding (still need to do after the `grouper` is available for python)

In [22]:
if revised_case_db.shape[0] <= len(revised_case_id):
    print(f'Based on the columns "case_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are {num_match_case_id} out of {len(revised_case_id)} revised cases from DtoD that can be matched with the bfs database data for {file_info.hospital_name_db} {file_info.year}' )
else:
    print(f'Based on the columns "case_id" and "Patient_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are {num_match_case_patient_id} out of {len(revised_case_id)} revised cases from DtoD that can be matched with the bfs database data for {file_info.hospital_name_db} {file_info.year}' )

Based on the columns "case_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are 55 out of 78 revised cases from DtoD that can be matched with the bfs database data for Hirslanden Zurich 2019


In [23]:
print('All the normalized revised cases are exported to here: {}'.format(output_path))

All the normalized revised cases are exported to here: /home/jovyan/work/src/revised_case_normalization/normalized_revision_data/Hirslanden_Zurich_2019.csv
